<h2><center>Η συμβολή της Διεσπαρμένης Παραγωγής στη ρύθμιση τάσης</center></h2>


<p>Τα  σύγχρονα  συστήματα  διανομής  έχουν  σχεδιαστεί  για να  διανέμουν το ρεύμα στους καταναλωτές, με  τη ροή της  πραγματικής και  της  άεργου  ισχύος να είναι  πάντα  από  το υψηλότερο προς στο  χαμηλότερο  επίπεδο  τάσης.<p> 
<p>Ωστόσο,  με τη σημαντική  διείσδυση  της Διεσπαρμένης Παραγωγής (ΔΠ), η ροή της ισχύος μπορεί να αντιστραφεί με αποτέλεσμα την ανύψωση τάσης. Έτσι, σε ορισμένες περιπτώσεις, η τάση μπορεί να ξεπεράσει το  ανώτατο  όριο.<p>

<div>
<img src="https://i.ibb.co/Vt7G5cr/Screenshot-from-2020-06-24-18-57-44.png" alt="Screenshot-from-2020-06-24-18-57-44" border="0" width="500"/>
</div>

<center> Σχήμα 1:Δίκτυο με διείσδυση ΔΠ</center>

Στο Σχήμα 1 φαίνεται το φορτίο με ενεργό και άεργο ισχύ ($P_L$ , $Q_L$) και η μονάδα ΔΠ με ενεργό και άεργο ισχύ ($P_{DG}$ , $Q_{DG}$) που συνδέονται στο δίκτυο μέσω γραμμής ωμικής αντίστασης $R$ και επαγωγικής αντίδρασης $X$.

$V_G$ είναι η τάση του δικτύου και $V_{PCC}$ η τάση του ζυγού φορτίου.

$P_{PCC}$ και $Q_{PCC}$ είναι η συνολική ενεργός και άεργος ισχύς που ρέει από τον ζυγό φορτίου.

Στην παρακάτω προσομοίωση θα μελετήσουμε τη συμβολή της ΔΠ στη ρύθμιση του φαινομένου της ανύψωσης τάσης. 

* Θα δούμε ότι η ανάστροφη ενεργού ισχύος από μια μονάδα ΔΠ ($P_{DG}$) στη γραμμή διανομής τείνει να αυξήσει την τάση στο σημείο σύνδεσης της μονάδας ΔΠ ($V_{PCC}$). 
* Η έγχυση αέργου ισχύος από τη μονάδα ΔΠ ($Q_{DG}$), μέσω της ρύθμισης του χωρητικού συντελεστή ισχύος ($\cosφ$), αυξάνει περεταίρω την τάση,  
* ενώ αντίθετα η απορρόφηση άεργου ισχύος από τη μονάδα ΔΠ ($Q_{DG}$), μέσω της ρύθμισης του επαγωγικού συντελεστή ισχύος ($\cosφ$) βοηθάει στον περιορισμό της ανύψωσης τάσης.

Στο παρακάτω τμήμα κώδικα δηλώνονται οι απαραίτητες βιβλιοθήκες της Python για την εκτέλεση του πειράματος. Χάρην συντομίας παρουσιάζουμε αυτό το κελί αναδιπλωμένο, αλλά το περιεχόμενό του μπορεί να εμφανιστεί πατώντας το βελάκι δεξιά ή αριστερά από το σχόλιο.  

In [1]:
# Imports
import matplotlib.pylab as plt
import pandas
import math

import numpy as np
import time
import pylab as pl
from IPython import display

import matplotlib.dates as md
from datetime import datetime
import datetime as dt

import threading

import ipywidgets as widgets
from ipywidgets import IntSlider
from ipywidgets import FloatSlider

from IPython.display import clear_output
from IPython.display import display

from ipywidgets import interactive, interactive_output
from ipywidgets import HBox

# Enable inline plotting
%matplotlib inline

Στη συνέχεια ορίζονται οι μεταβλητές που θα χρησιμοποιηθούν.
* $R= 2.35Ω$, $Χ = 1.71Ω$, $P_L= 300kW$, $Q_L= 227kVAr$ 
* Η τάση του δικτύου θεωρείται ότι έχει την ονομαστική της τιμή $V_{G}= 230kV$

In [2]:
global Pdg
global Qdg
global Pl
global Ql
global Ppcc
global Qpcc
global past_pf_in 
global past_pf_ca 

R = 2.35  
X = 1.17
Pl = 300
Ql = 227
Vg = 230 

<p>Το παρακάτω τμήμα κώδικα δημιουργεί και εμφανίζει 3 διαδραστικά στοιχεία εισόδου χρήστη για την χειροκίνητη ρύθμιση των τιμών της ενεργού ισχύος της μονάδας ΔΠ ($P_{DG}$), του επαγωγικού και του χωρητικού συντελεστή ισχύος ($\cosφ$).<p>
<p>Μέσα σε ένα while loop υπολογίζονται κάθε στιγμή οι τιμές των $V_{PCC}$,$P_{PCC}$ και $Q_{PCC}$ από τις παρακάτω σχέσεις και οι αυξομοιώσεις τους παρουσιάζονται σε τρεις γραφικές παραστάσεις σε συνάρτηση με τον χρόνο.<p> 

Η γωνία $φ$ υπολογίζεται από τη σχέση:

$$φ= \cos^{-1}φ  \quad (1) $$

Από τις σχέσεις:

$$P_{DG} = V_{PCC}\cdot I \cdot \cosφ \quad (2α) $$

$$Q_{DG} = V_{PCC}\cdot I \cdot \sinφ \quad (2β) $$

προκύπτει η σχέση:

$$Q_{DG} = P_{DG}\cdot \tanφ \quad (2) $$

H ενεργός και άεργος ισχύς της μονάδας ΔΠ δίνεται από τις σχέσεις:

$$P_{PCC} = P_{DG} - P_L \quad (3) $$

$$Q_{PCC} = - Q_{DG}- Q_L \quad (4α) $$

$$Q_{PCC} = Q_{DG}- Q_L \quad (4β) $$

<pr>Η σχέση $(4α)$ ισχύει όταν η μονάδα ΔΠ λειτουργεί με επαγωγικό ΣΙ, οπότε απορροφά άεργο ισχύ. <pr>
<pr>Αντίστοιχα η σχέση $(4β)$ ισχύει όταν η μονάδα λειτουργεί με χωρητικό ΣΙ, όπότε παράγει άεργο ισχύ.<pr>

Επομένως: 

H μεταβολή της τάσης υπολογίζεται προσεγγιστικά ως:

$$ΔV \approx \frac{P_{PCC}\cdot R + Q_{PCC}\cdot Χ}{V_{PCC}}\approx \frac{P_{PCC}\cdot R + Q_{PCC}\cdot Χ}{V_{G}}  \quad (5)$$

Από τη σχέση $(6)$ υπολογίζεται η τάση του ζυγού φορτίου:

$$ΔV = V_{PCC} - V_{G} \Rightarrow  V_{PCC}= ΔV + V_{G} \quad (6) $$ 

Ακολουθώντας τα παρακάτω βήματα μπορούμε να παρατηρήσουμε την επίδραση της μόνάδας ΔΠ στη ρύθμιση τάσης:
1. Αρχικά η ενεργός ισχύς που παράγει η ΔΠ είναι $P_{DG} =300 KW$ και $cosφ=1$ άρα $Q_{DG} =0$. 

2. Αυξάνουμε την ενεργό ισχύ στην τιμή $P_{DG} =350 KW$ διατηρώντας το $cosφ$ σταθερό και παρατηρούμε την αύξηση της τάσης $V_{PCC}$, λόγω της αντίθετης ροής ενεργού ισχύος πάνω στη γραμμή διανομής.

3. Διατηρώντας σταθερή την ενεργό ισχύ και ρυθμίζοντας το $cosφ=0.9$ χωρητικό παρατηρούμε ότι η τάση $V_{PCC}$ αυξάνεται περεταίρω, λόγω της έγχυσης αέργου ισχύος από τη μονάδα ΔΠ.

4. Διατηρώντας σταθερή την ενεργό ισχύ και ρυθμίζοντας το $cosφ=0.9$ επαγωγικό παρατηρούμε μείωση της τάσης $V_{PCC}$, καθώς απορροφάται άεργος ισχύς από τη μονάδα ΔΠ.    


In [3]:
# Slider variables
w = widgets.IntSlider(value=300, description='Pdg', min=200,max=400,step=50)
pf_in = FloatSlider(value=1, description='cosφ επαγ.', min=0.8,max=1,step=0.1)
pf_ca = FloatSlider(value=1, description='cosφ χωρ.', min=0.8,max=1,step=0.1)

ui = widgets.HBox([w, pf_in, pf_ca])

def f(w, pf_in, pf_ca):
    display((w, pf_in, pf_ca))


def on_value_change_Pdg(change):
    Pdg = change['new']
    
def on_value_change_pf_in(change):
    pf_in = change['new']
    
def on_value_change_pf_ca(change):
    pf_ca = change['new']    
            
w.observe(on_value_change_Pdg, names='value')
pf_in.observe(on_value_change_pf_in, names='value')
pf_ca.observe(on_value_change_pf_ca, names='value')   

past_pf_in = pf_in.value
past_pf_ca = pf_ca.value

        
# Plot 
def plot(stime,stemp,stemp1,stemp2):   
    f, axarr = plt.subplots(3, sharex=True, figsize=(7,7))
    axarr[0].step(stime,stemp)
    axarr[0].set_ylim(227, 231)
    
    axarr[1].step(stime,stemp1,'r')
    axarr[1].set_ylim(-100,150)
    
    axarr[2].step(stime,stemp2,'g')
    axarr[2].set_ylim(-500,0)

    axarr[0].set_ylabel("Vpcc(KV)")
    axarr[1].set_ylabel("Ppcc(KW)")
    axarr[2].set_ylabel("Qpcc(KVar)")
    
    axarr[2].set_xlabel("Time")
    
    axarr[0].set_title("Voltage")
    axarr[1].set_title("Active Power")
    axarr[2].set_title("Reactive Power")
    
    clear_output(wait=True)
    display(plt.gcf())
    plt.gcf().clear()
    time.sleep(2)

Output()

In [4]:
out = widgets.interactive_output(f, {'w': w, 'pf_in': pf_in, 'pf_ca': pf_ca})

display(ui, out)


def train():
    stime = np.array([])  
    stemp = np.array([])
    stemp1 = np.array([])
    stemp2 = np.array([])
    
    
    while True:
        try: 
            # Αν cosφ επαγωγικός ή cosφ = 1
            if (pf_in.value ==1 and pf_ca.value ==1) or (pf_in.value != past_pf_in):
                φ = math.acos(pf_in.value)   # (1)   
                Pdg = w.value
                Qdg =Pdg*math.tan(φ)         # (2)
                Qpcc = - Qdg - Ql            # (4α)
                
            # Αν cosφ χωρητικός
            elif pf_ca.value != past_pf_ca: 
                φ = math.acos(pf_ca.value)   # (1)
                Pdg = w.value
                Qdg =Pdg*math.tan(φ)         # (2)
                Qpcc = Qdg - Ql              # (4β)

            
            Ppcc = Pdg - Pl                  # (3)    
            ΔV = (Ppcc*R + Qpcc*X)/Vg        # (5)
            Vpcc = Vg + ΔV                   # (6)
            
            print("Vpcc =", Vpcc, "kV", ", Ppcc =", Ppcc, "kW", ", Qpcc =", Qpcc, "KVAr")
            
            newTime = datetime.now()
            temp = Vpcc
            stime = np.append(stime,newTime)    # Store the values
            stemp = np.append(stemp,temp)
            stemp1 = np.append(stemp1,Ppcc)
            stemp2 = np.append(stemp2,Qpcc)
            plot(stime,stemp,stemp1,stemp2)
            
        except KeyboardInterrupt:
            break  

try:
    thread = threading.Thread(target=train, args=())
    thread.start()
except:
    print("error")

Output()

Vpcc = 228.8452608695652 kV , Ppcc = 0 kW , Qpcc = -227.0 KVAr
